# Imports

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

# Device configuration


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# 1. Data preprocessing


In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),              # Resize to 224x224
    transforms.Grayscale(num_output_channels=3),# Convert 1 channel to 3 channels
    transforms.ToTensor(),                      # Convert to tensor
    transforms.Normalize((0.5,), (0.5,))         # Normalize (simple mean/std)
])

train_dataset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset  = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=64, shuffle=False)


# 2. Load VGG16 model and modify the classifier


In [ ]:
model = models.vgg16(pretrained=True)
model.classifier[6] = nn.Linear(4096, 10)  # Fashion MNIST has 10 classes
model = model.to(device)

# 3. Loss and optimizer


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# 4. Training function


In [ ]:
def train_model(model, epochs=5):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")


# 5. Testing function


In [ ]:
def test_model(model):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Test Accuracy: {100 * correct / total:.2f}%")

# 6. Run training and testing

In [ ]:

train_model(model, epochs=5)
test_model(model)